# Problem 1
We're just looking for something to indicate that you read Shannon's paper and got _something_ out of it. If you did this, full credit.

# Problem 2: Scraping, Entropy and ICML papers

### 2.0: Scraping text from all .pdfs

In [9]:
# Import block
!pip install pdfminer.six

import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import urllib
import io
import string
import csv
import joblib
import re
from tqdm import tqdm

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

from nltk import word_tokenize
from nltk.corpus import stopwords

import pandas as pd
import numpy as np

# Suppress warnings for readability
import warnings
warnings.filterwarnings("ignore")


In [2]:
# Build utilities to convert pdfs to strings 

url = "http://proceedings.mlr.press/v70/"

response = requests.get(url)
soup= BeautifulSoup(response.text, "html.parser")    
all_pdf_urls = [urljoin(url,link['href']) for link in soup.select("a[href$='.pdf']")]

def convert_online_pdf_to_string(pdfurl):
    # General method using pdfminer to convert pdf FILE to string
    file = io.BytesIO(urllib.request.urlopen(pdfurl).read())
    output_string = io.StringIO()
    parser = PDFParser(file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)
    return(output_string.getvalue())

def convert_link_pdf_to_string(link):
    # Process the html link to the true url for downloading
    filename = link['href'].split('/')[-1]
    pdfstring = ''
    if 'sup' in filename:
        return pdfstring
    pdfurl = urljoin(url,link['href'])
    try:
        pdfstring = convert_online_pdf_to_string(pdfurl)
    except:
        print('Cannot parse {}'.format(filename))
    return pdfstring



In [5]:
#parse all pdfs to string using the above methods
all_pdf_strings = []
index = 0
count = 0
for link in tqdm(soup.select("a[href$='.pdf']")):
    filename = link['href'].split('/')[-1]
    if 'supp' in filename:
        continue
    pdfurl = urljoin(url,link['href'])
    try:
        pdfstring = convert_online_pdf_to_string(pdfurl)
        all_pdf_strings.append(pdfstring)
#         print(str(index)+' : '+ filename)
        index += 1
    except:
        print('Cannot parse {}'.format(filename))

100%|██████████| 722/722 [21:31<00:00,  1.79s/it]  


In [10]:
# Preproessing Text:
# There are several ways to preprocess text. 
# Many Python libraries (nltk, scikit learn, gensim,...)
# alredy have built-in functions to do it 

def preprocess(docs):
    
    #remove breaklines, convert to lowercase
    docsProc = [str.replace(docs[i], '\n', ' ') for i in range(len(docs))]
    docsProc = [u.lower() for u in docsProc]

    #remove punctuation
    docsProc = [''.join(c for c in doc if c not in string.punctuation) for doc in docsProc]

    #remove numbers
    docsProc = [re.sub("\d+", " ", doc) for doc in docsProc]

    #trim whitespace
    docsProc = [re.sub( '\s+', ' ', doc ).strip() for doc in docsProc]

    # Vectorize text by using bag of words. 
    # Notice that this function has parameters to do some of the preprocessing above...
    from sklearn.feature_extraction.text import CountVectorizer

    #read parameters of this function for text preprocessing... stopwords, lowercase, etc
    vectorizer = CountVectorizer(stop_words='english', lowercase =True)
    X = vectorizer.fit_transform(docsProc)

    # One way to easily explore frequency terms is converting X to a DAtaframe
    return pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())

# Convert text to Bag of Words data frame
dtm = preprocess(all_pdf_strings)



### 2.1: Collecting 10 most common words

In [11]:
def top_k_words(df, k):
    
    # Sum the frequency of each word over all documents
    highest_freq = df.sum(axis=0) 
    highest_freq = highest_freq.sort_values(ascending=False) 
    
    return highest_freq[0:k]

print("Top 10 words:")
top_k_words(dtm, 10)

Top 10 words:


cid          33434
al           14891
et           14344
learning     10952
model         7851
data          7388
algorithm     7110
set           5942
function      5471
using         5301
dtype: int64

### 2.2: Estimating Entropy of $Z$

Let $Z$ be a randomly selected word in a randomly selected ICML paper.  Estimate the entropy of $Z$.

To estimate the entropy of $Z$ we first need to estimate the distribution of $Z$. The wording describing the distribution of $Z$ was slightly ambiguous. Be sure to understand the difference between these two distributions:

**Distribution 1**
- Probability of selecting word $z$ is the number of times $z$ appears in the corpus divided by the total number of words in the corpus.

**Distribution 2**

- Each PDF is randomly selected with probability $P(\text{paper} = i) = \frac{1}{\# \text{papers}}$.

- Let $N_i$ be the number of different words in the paper $i$.  Then P(Z=z | paper = i) can be estimated by the relative frequency of the word $z$ in paper $i$.

$$ P(Z=z |\text{paper} = i) = \frac{\text{absolute frequeny of $z$ in paper $i$}}{N_i} $$

Using the law of total probability, the marginal distribution of Z is
$$ P(Z=z) = \sum_{i \in [\# \text{papers}]} P(Z=z |\text{paper} = i) P(\text{paper} = i) = \frac{1}{\# \text{papers}} \sum_{i \in [\# \text{papers}]} P(Z=z |\text{paper} = i)$$

**Computing Entropy**
Given a function to compute the probability of word $z$, denoted $p_z$, computing the entropy $H(Z)$ is done using the formula 
$$H(Z) = \sum_{z\in \text{Supp}(Z)} -p_z\cdot \log(p_z)$$

In [12]:
# Probability of each paper i, P(paper=i)
prob_of_paper = 1.0/dtm.shape[0]
# Total number of words in paper i, N_i
total_words_per_paper = dtm.sum(axis=1)
# Relative Frequency of each word in each paper, P(Z=z|paper=i)
relative_freqs_per_paper = dtm.divide(total_words_per_paper, axis=0)

# Marginal Probability for each word P(Z=z)
P_z = prob_of_paper*relative_freqs_per_paper.sum(axis=0)

# Entropy
entropy = -P_z.multiply(np.log(P_z)).sum()
print(entropy)



8.606904135491124


### 2.3 Synthesizing a random paragraph

In [13]:
# Produce a paragraph of length l
# param l: lenghth of the paragraph.
# param p: words distribution
def produce_paragraph(l, p):
    wordsIndex = [np.random.multinomial(1,p,1).argmax() for i in range(l)]
    return " ".join(p.index.values[wordsIndex])
produce_paragraph(100, P_z)

'frames increasingly inchi output oy perturbed section klaus priors process tsybakov instead takahashi baselines expect temporal width samples ab wγst methods mcsherry forcement similar cid slightly min develop vs note cid deﬁned alternative cost learning algorithm fc fairly block journal varying gdeltraw total feedback weights property iteration et fi ηl methods generalizing arxiv ut turning elements mouchere testing largescale analysis individual variance term ﬁt transcends infer layer respectively benign sharp demonstrate apply threshold ˆwx agent srevengelogscale paper zcdp acm possible depth set graph non oracle tens factor small approxi iht leaves data instruction cdmcboost cid experiments corre spread dropout huffman'

# Problem 3: More on Kaggle Advanced Regression


### 3.1 Forum Posts:
I found the following forum posts to be thorough, informative, and quite helpful: 
- [https://www.kaggle.com/faressayah/linear-regression-house-price-prediction](https://www.kaggle.com/faressayah/linear-regression-house-price-prediction)
- [https://www.kaggle.com/juliencs/a-study-on-regression-applied-to-the-ames-dataset](https://www.kaggle.com/juliencs/a-study-on-regression-applied-to-the-ames-dataset)
- [https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard)

A note on grading: We're only looking for some reasonably good performance on the dataset and some minor discussion on your methods. I'll copy from some of the forum posts linked above, but ideally you'll have played with this yourself. 

In [14]:
# Import block 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, skew #for some statistics

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import sklearn.metrics as metrics
#import xgboost as xgb
#import lightgbm as lgb
%matplotlib inline


### 3.0 Data Preprocessing

In [15]:
# Data loading
train = pd.read_csv('train.csv') # .csv's should be in the local directory 
test = pd.read_csv('test.csv')

#Save and drop the 'Id' columns
train_ID = train['Id']
test_ID = test['Id']
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)


train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [16]:
# Data cleaning/pre-processing

# Drop outliers..
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

# Log1p the target column for skewness
train["SalePrice"] = np.log1p(train["SalePrice"])

# Concatenate into a single data frame to do transformations only once...
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)

# Clean missing data 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio
PoolQC,99.691464
MiscFeature,96.400411
Alley,93.212204
Fence,80.425094
FireplaceQu,48.680151
LotFrontage,16.660953
GarageFinish,5.450806
GarageYrBlt,5.450806
GarageQual,5.450806
GarageCond,5.450806


In [17]:
# Impute missing values. See justifications from 
# https://www.kaggle.com/bucket/stacked-regressions-top-4-on-leaderboard/edit

# FillNA with None...
for feature in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 
                'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                'MasVnrType', 'MSSubClass', 'FireplaceQu']:
    all_data[feature] = all_data[feature].fillna('None')
    
# FillNA with 0
for feature in ['GarageYrBlt', 'GarageArea', 'GarageCars', 
                'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                'MasVnrArea']:
    all_data[feature] = all_data[feature].fillna(0)

# FillNA with the mode
for feature in ['MSZoning', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd','SaleType']:
    all_data[feature] = all_data[feature].fillna(all_data[feature].mode()[0])

    
# Weird cases: 
all_data["Functional"] = all_data["Functional"].fillna("Typ")
try: # for idempotency
    all_data = all_data.drop(['Utilities'], axis=1)
except: 
    pass

#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

# And finally check missing_data 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head() # Should be empty here if we did our job right


,Missing Ratio


In [18]:
# Categorical features

# Numericals that are actually categorical
actually_cat = ['MSSubClass', 'OverallCond', 'YrSold', 'MoSold']
for feature in actually_cat: 
    all_data[feature] = all_data[feature].astype(str)
    
# Label encoding for things that have some ordering 

from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))


Shape all_data: (2917, 78)


In [19]:
# One final hand-designed feature 
# Adding total sqfootage feature 
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']


In [20]:
# Fix skewness 
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})

skewness


Skew in numerical features: 



,Skew
MiscVal,21.939672
PoolArea,17.688664
LotArea,13.109495
LowQualFinSF,12.084539
3SsnPorch,11.372080
LandSlope,4.973254
KitchenAbvGr,4.300550
BsmtFinSF2,4.144503
EnclosedPorch,4.002344
ScreenPorch,3.945101


In [21]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 59 skewed numerical features to Box Cox transform


In [22]:
# Add dummies for categoricals
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2917, 220)


In [23]:
# and finally collect all train/test to use forreals
train = all_data[:ntrain]
test = all_data[ntrain:]

#### NOTE: Because we want to actually evaluate for overfitting later, 
#          let's do another split here

def shuffle_split_data(X, y, pct=70):
    # Shamelessly stolen from SO: 
    # https://stackoverflow.com/questions/35932223/writing-a-train-test-split-function-with-numpy
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, pct)

    X_train = X[split]
    y_train = y[split]
    X_test =  X[~split]
    y_test = y[~split]

    return X_train, y_train, X_test, y_test

train, y_train, holdout, y_holdout = shuffle_split_data(train, y_train)


### 3.2 Best model. 
Here we'll use the super simple stacked regressors from one of the forum posts, but you can try many other things

In [24]:
# Hardcoding the CV code
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [25]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [26]:
score = rmsle_cv(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.1134 (0.0049)

ElasticNet score: 0.1134 (0.0050)

Kernel Ridge score: 0.1178 (0.0059)

Gradient Boosting score: 0.1172 (0.0044)



In [27]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [28]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.1095 (0.0052)



### 3.3 Over and Underfitting
We'll primarily be looking for:
1. Underfitting: super simple model class and therefore high train AND test error 
2. Overfitting: complex model, very low train error, but high test error

In [29]:
def print_evaluate(model):  
    
    for data, true, id_ in [(train, y_train, 'Training Data'), 
                           (holdout, y_holdout, 'Holdout Data')]:
        predicted = model.predict(data)
        mae = metrics.mean_absolute_error(true, predicted)
        mse = metrics.mean_squared_error(true, predicted)
        rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
        r2_square = metrics.r2_score(true, predicted)
        print(id_)
        print('MAE:', mae)
        print('MSE:', mse)
        print('RMSE:', rmse)
        print('R2 Square', r2_square)
        print('__________________________________')

In [30]:
# Super simple model -- linear regression with no intercept
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression(fit_intercept=False)
lin_reg.fit(train,y_train)

print_evaluate(lin_reg)

Training Data
MAE: 0.06312826016446431
MSE: 0.007708417506406449
RMSE: 0.08779759396706979
R2 Square 0.9528774355754969
__________________________________
Holdout Data
MAE: 0.09624216888111325
MSE: 0.028339306464988053
RMSE: 0.16834282421590785
R2 Square 0.8117343269004165
__________________________________


In [31]:
# Decision tree with random splitting and really big depth 
from sklearn.tree import DecisionTreeRegressor 

dtree = DecisionTreeRegressor(splitter='random', max_depth=1000)
dtree.fit(train, y_train)
print_evaluate(dtree)

Training Data
MAE: 0.0
MSE: 0.0
RMSE: 0.0
R2 Square 1.0
__________________________________
Holdout Data
MAE: 0.15288021441755734
MSE: 0.047101500815335824
RMSE: 0.21702880181057957
R2 Square 0.6870919983184733
__________________________________
